In [1]:
import pandas as pd
import numpy as np
import pickle
from math import sqrt

import warnings
import scipy.stats as stats
import xgboost as xgb

import shap

import torch
from torch import nn, optim

import category_encoders as ce

from tqdm import tqdm

import gc

import time

In [2]:
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

In [3]:
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer

from spark_sklearn import GridSearchCV

The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.


In [4]:
import lightgbm
from lightgbm import Dataset 
from lightgbm import train
from lightgbm import cv
from datetime import datetime

# LightGBM

In [5]:
#levanto data sets
df = pd.read_csv("../../machine-learning/cleanedData.csv")
y = df.precio

X = pd.read_csv("../../machine-learning/xgboost-x.csv")

In [6]:
aux = X.join(y)
aux = aux.dropna()
aux = aux.sample(n=5000, random_state=42)

In [7]:
y_aux = aux.precio
#y_aux = np.log1p(y_aux)# applying log transformation here

X_aux = aux.drop(['precio'], axis=1, inplace=False)
X_aux.drop(['Unnamed: 0'], axis=1, inplace=True)
X_aux.drop(['id'], axis=1, inplace=True)

y_test = pd.DataFrame(y_aux)

In [8]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X_aux, y_aux, test_size=test_size, random_state=seed)

In [9]:
def save_prediction(y_test, ids, model):
		final_pred = y_test

#		ids = self.df_test['id'].values
		try:
			os.mkdir('predictions')
		except:
			pass


		submit = pd.DataFrame({'id':ids,'target':final_pred})
		submit.to_csv('../../machine-learning/predictions/submit-'+model+'.csv',index=False)

In [10]:
def timer(start_time=None):
		if not start_time:
			start_time = datetime.now()
			return start_time
		elif start_time:
			thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
			tmin, tsec = divmod(temp_sec, 60)
			print('Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [11]:
def prepare_data():
    lgb_train = Dataset(X_train,y_train)#,free_raw_data=False)
    lgb_eval = Dataset(X_test,y_test)#,free_raw_data=False)

    return lgb_train,lgb_eval

In [12]:
def print_metrics(y_test, y_test_pred):
    # mean_absolute_error
		mae = mean_absolute_error(y_test, y_test_pred)
		print("")
		print("mean_absolute_error (MAE): {0:.5f}".format(mae)) 

		rms = sqrt(mae)
		print("")
		print("root mean_absolute_error (RMSE): {0:.5f}".format(rms))     
    
    #calculating accuracy
#		accuracy_lgbm = accuracy_score(y_test_pred, y_test)
#		print("LightGBM accuracy score: {0:.5f}".format(accuracy_lgbm)) #ValueError: continuous is not supported
#Is a classification or ranking metric, not a regression metric. So it doesn't accept continuous y.
#https://github.com/scikit-learn/scikit-learn/issues/6592
#		print("")
    
    #calculating roc_auc_score for light gbm.
#		auc_lgbm = roc_auc_score(y_test,y_test_probs)
#		print("LightGBM auc score: {0:.5f}".format(auc_lgbm))
#		print("")
    

In [13]:
def train_model_nocv(lgb_train,lgb_eval,params):

		print('Start training...')
		start_time = timer()

		booster = train(
				params=params,
				train_set=lgb_train,
				valid_sets=[lgb_eval],
				verbose_eval=100,
			)

		"""booster = cv(
			params=params,
			train_set=lgb_train,
			)"""

		timer(start_time)

		print("")
		print('Saving model into a pickle')
		try:
			os.mkdir('pickles')
		except:
			pass

		with open('../../machine-learning/pickles/lightgbm.pkl','wb') as f:
			pickle.dump(booster, f)

		print('Making prediction and saving into a csv')
		y_test_pred = booster.predict(X_test)


		return y_test_pred

In [ ]:
For Better Accuracy

Use large max_bin (may be slower)
Use small learning_rate with large num_iterations
Use large num_leaves (may cause over-fitting)
Use bigger training data
Try dart

In [47]:
params = {'num_leaves':150,
          'max_depth':7,
          'learning_rate':.02,
          'max_bin':150,
          'num_iterations':1000 #default = 100          
         }
params['metric'] = ['auc', 'mae', 'rmse']

lgb_train,lgb_eval = prepare_data()
y_test_pred = train_model_nocv(lgb_train, lgb_eval, params)
print_metrics(y_test, y_test_pred)

Start training...


Found `num_iterations` in params. Will use it instead of argument


[100]	valid_0's auc: 1	valid_0's l1: 595242	valid_0's rmse: 932438
[200]	valid_0's auc: 1	valid_0's l1: 544921	valid_0's rmse: 885608
[300]	valid_0's auc: 1	valid_0's l1: 535553	valid_0's rmse: 879763
[400]	valid_0's auc: 1	valid_0's l1: 532953	valid_0's rmse: 879574
[500]	valid_0's auc: 1	valid_0's l1: 531507	valid_0's rmse: 880969
[600]	valid_0's auc: 1	valid_0's l1: 532153	valid_0's rmse: 884970
[700]	valid_0's auc: 1	valid_0's l1: 531591	valid_0's rmse: 886741
[800]	valid_0's auc: 1	valid_0's l1: 531662	valid_0's rmse: 888299
[900]	valid_0's auc: 1	valid_0's l1: 532192	valid_0's rmse: 889796
[1000]	valid_0's auc: 1	valid_0's l1: 532953	valid_0's rmse: 892003
Time taken: 0 hours 0 minutes and 1.3 seconds.

Saving model into a pickle
Making prediction and saving into a csv

mean_absolute_error (MAE): 532952.90604

root mean_absolute_error (RMSE): 730.03624


In [ ]:
ids = X_test.index
save_prediction(y_test_pred, ids, 'LightGBM')

In [ ]:
#
#
PORQUE EL RMSE DE MI FUNCION print_metrics DA DISTINTO AL QUE IMPRIME lightgbm.train ???????
#
#

# LightGBM Random Search

In [50]:
params = {'num_leaves':[75, 100, 125, 150, 175],
          'max_depth':[2,4,6,8],
          'learning_rate':[.01, .05, .1, .015, .2],
          'max_bin':[50, 100, 150, 200],
          'num_iterations':[1000] #default = 100          
         }

In [51]:
clf = lightgbm.LGBMRegressor()
rs_clf = RandomizedSearchCV(clf, params, n_iter=40,
                            verbose=2, cv=3,
                            scoring='neg_mean_squared_error', random_state=42)

# MSE == neg_mean_squared_error

In [52]:
print("Randomized search..")
print("")
start_time = timer()
rs_clf.fit(X_aux, y_aux)
timer(start_time)

Randomized search..

Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV] num_leaves=175, num_iterations=1000, max_depth=4, max_bin=150, learning_rate=0.1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  num_leaves=175, num_iterations=1000, max_depth=4, max_bin=150, learning_rate=0.1, total=   0.7s
[CV] num_leaves=175, num_iterations=1000, max_depth=4, max_bin=150, learning_rate=0.1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
Found `num_iterations` in params. Will use it instead of argument


[CV]  num_leaves=175, num_iterations=1000, max_depth=4, max_bin=150, learning_rate=0.1, total=   0.7s
[CV] num_leaves=175, num_iterations=1000, max_depth=4, max_bin=150, learning_rate=0.1 


...

[CV]  num_leaves=100, num_iterations=1000, max_depth=2, max_bin=200, learning_rate=0.05, total=   0.5s
[CV] num_leaves=100, num_iterations=1000, max_depth=2, max_bin=200, learning_rate=0.05 


Found `num_iterations` in params. Will use it instead of argument


[CV]  num_leaves=100, num_iterations=1000, max_depth=2, max_bin=200, learning_rate=0.05, total=   0.5s


[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.7min finished
Found `num_iterations` in params. Will use it instead of argument


Time taken: 0 hours 1 minutes and 43.21 seconds.


In [ ]:
best_score = rs_clf.best_score_
best_params = rs_clf.best_params_
print("Best score: {}".format(best_score))
print("")
print("Best params: ")
for param_name in sorted(best_params.keys()):
    print('%s: %r' % (param_name, best_params[param_name]))

# LightGBM Grid Search

In [ ]:
ME TIRA EL ERRROR
TypeError: __init__() missing 1 required positional argument: 'sc'
QUE ES UN PARAMETRO DE LA GridSearchCV EN TEORIA... PERO EN LA DOCUMENTACION NO FIGURA 
Y GOOGLIE EL ERROR Y TAMPOCO LO ENCONTRE

### Intento pulir a mano el mejor set de parametros obtenido en la RandomSearchCV

In [ ]:
params = {'num_leaves':[100],
          'max_depth':[4],
          'learning_rate':[.05],
          'max_bin':[150],
          'num_iterations':[1000] #default = 100          
         }

In [ ]:
https://www.analyticsvidhya.com/blog/2017/06/which-algorithm-takes-the-crown-light-gbm-vs-xgboost/

For best fit

num_leaves : This parameter is used to set the number of leaves to be formed in a tree. 
    Theoretically relation between num_leaves and max_depth is num_leaves= 2^(max_depth). 
    However, this is not a good estimate in case of Light GBM since splitting takes place leaf wise rather
    than depth wise. Hence num_leaves set must be smaller than 2^(max_depth) otherwise it may lead to overfitting. 
    Light GBM does not have a direct relation between num_leaves and max_depth and hence the 
    two must not be linked with each other.

    min_data_in_leaf : It is also one of the important parameters in dealing with overfitting. 
        Setting its value smaller may cause overfitting and hence must be set accordingly. 
        Its value should be hundreds to thousands of large datasets.

    max_depth: It specifies the maximum depth or level up to which tree can grow.

In [ ]:
https://www.analyticsvidhya.com/blog/2017/06/which-algorithm-takes-the-crown-light-gbm-vs-xgboost/

For better accuracy

Use bigger training data

num_leaves : Setting it to high value produces deeper trees with increased accuracy but lead to overfitting. 
    Hence its higher value is not preferred.

max_bin : Setting it to high values has similar effect as caused by increasing 
    value of num_leaves and also slower our training procedure.

In [ ]:
# CONTINUAR !!!!!!!
# CONTINUAR !!!!!!!
# CONTINUAR !!!!!!!

# AdaBoost

In [ ]:
fijarme si documente otros !!!!

In [39]:
open('../../machine-learning/pickles/lightgbm.pkl','wb')

<_io.BufferedWriter name='../../machine-learning/pickles/lightgbm.pkl'>